In [1]:
## IMPORT MODULES
import pandas as pd
import numpy as np

In [2]:
## IMPORT PLOTTING MODULES
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import matplotlib as mpl
from matplotlib import cm
%matplotlib widget

In [3]:
## FULL DATAFRAME
df_raw = pd.read_csv("NS_2-Policy_Summary.csv")
df_raw = df_raw.drop(df_raw[
    (df_raw['landing_rate']<=0.1) & 
    (df_raw['vz_d']>=2.75)
    ].index)

df_raw = df_raw.drop(df_raw[
    (df_raw['landing_rate']<=0.1) & 
    (df_raw['vx_d']>=1.0)
    ].index)



df_raw_avg = df_raw.groupby(['vz_d','vx_d']).agg([np.mean,np.std]).reset_index()

df = df_raw
df['My_d'] = df['My_d'].apply(lambda x: np.abs(x))
df = df.drop(df[df['My_d']>=7.7].index)

df = df.dropna()

## AVERAGED DATAFRAME
df_avg = df.groupby(['vz_d','vx_d']).agg([np.mean,np.std]).reset_index()

In [4]:
df

,vz_d,vx_d,trial_num,landing_rate,RREV_threshold,G1,RREV_sig,G1_sig,RREV_trigger,OF_y,My_d,impact_eul,impact_tdelta,alpha_mu,alpha_sigma,mu_ini,sigma_ini
4,1.75,0.00,4,0.166667,4.75,5.48,0.01,0.01,5.342250,0.000000,5.485500,-103.875433,0.193500,[0. 0.],[0. 0.],[3.4 4.9],[1.5 1.5]
5,1.75,0.00,5,0.041667,4.71,5.40,0.00,0.04,4.942000,0.000000,5.424000,-163.300834,0.226000,[0. 0.],[0. 0.],[6.06 3.16],[1.5 1.5]
6,1.75,0.25,0,0.125000,4.77,6.28,0.03,0.00,4.883000,-0.621000,6.281000,-160.017116,0.212000,[0. 0.],[0. 0.],[3.33 5.84],[1.5 1.5]
10,1.75,0.25,4,0.117647,5.05,2.93,0.00,0.00,5.681500,-0.722000,2.931000,-95.386142,0.235000,[0. 0.],[0. 0.],[2.08 2.12],[1.5 1.5]
11,1.75,0.25,5,0.066667,5.04,2.95,0.00,0.00,5.732000,-0.729000,2.950000,-95.356706,0.232000,[0. 0.],[0. 0.],[3.78 2.44],[1.5 1.5]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
687,4.00,2.75,0,1.000000,3.09,7.05,0.00,2.74,3.306500,-2.166500,7.220500,-117.505141,0.414000,[0. 0.],[0. 0.],[2.1 5.48],[1.5 1.5]
688,4.00,2.75,1,0.714286,5.30,6.64,0.01,0.01,5.751733,-3.742467,6.643333,-130.760512,0.189467,[0. 0.],[0. 0.],[4.27 5.64],[1.5 1.5]
689,4.00,2.75,2,0.916667,5.03,6.53,0.06,0.25,5.593409,-3.649273,6.652909,-129.975835,0.193364,[0. 0.],[0. 0.],[1.58 6.99],[1.5 1.5]
690,4.00,2.75,3,0.913043,3.99,2.13,0.00,0.05,4.211286,-2.742381,2.133095,-137.563149,0.307619,[0. 0.],[0. 0.],[4.75 2.4 ],[1.5 1.5]


In [5]:

## LANDING RATE DATA
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")
ax.scatter(df_raw['vx_d'],df_raw['vz_d'],df_raw['landing_rate'])

ax.set_xlabel('vx_d')
ax.set_ylabel('vz_d')
ax.set_zlabel('Landing Rate')
ax.set_title('Landing Rate (Raw Data)')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:

## AVG LANDING RATE SURFACE
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")
cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=0,vmax=1)
pcm = ax.plot_trisurf(df_raw_avg['vx_d'],df_raw_avg['vz_d'],df_raw_avg['landing_rate','mean'],cmap = cmap)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Landing %")
df
ax.set_zlim(0,1)

ax.set_xlabel('vx_d')
ax.set_ylabel('vz_d')
ax.set_zlabel('Landing Rate')
ax.set_title('Avg Landing Rate for Final 3 Episodes')


plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
## RREV Threshold vs IC
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

ax.scatter(df['vx_d'],df['vz_d'],df['RREV_threshold'])


ax.set_xlim(0,2.75)
ax.set_ylim(1.0,4.5)


ax.set_xlabel('Vx_d')
ax.set_ylabel('Vz_d')
ax.set_zlabel('RREV_threshold')
ax.set_title('RREV_thr vs IC - (Raw Data)')


plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:

## Avg RREV_thr vs IC
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=4,vmax=5)
pcm = ax.plot_trisurf(df_avg['vx_d'],df_avg['vz_d'],df_avg['RREV_threshold','mean'],cmap=cmap)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="RREV_threshold")

ax.set_xlim(0,2.75)
ax.set_ylim(1.0,4.5)
ax.set_zlim(3,6)

ax.set_xlabel('vx_d')
ax.set_ylabel('vz_d')
ax.set_zlabel('RREV_threshold')

ax.set_title('Avg RREV_thr vs IC')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=4,vmax=5)
pcm = ax.scatter(df['vx_d'],df['vz_d'],df['impact_tdelta'])
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="RREV_threshold")


ax.set_xlabel('vx_d')
ax.set_ylabel('vz_d')
ax.set_zlabel('Delta_t [s]')

ax.set_title('Time Rotating vs IC (Raw Data)')

ax.set_zlim(0,0.5)




plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")


cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=4)

pcm = ax.scatter(df['RREV_trigger'],df['landing_rate'],df['impact_tdelta'],c=df['vz_d'],cmap=cmap)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Vz_d')


ax.set_xlabel('RREV_trigger')
ax.set_ylabel('Landing_rate')
ax.set_zlabel('Delta_t [s]')

ax.set_title('Time Rotating vs RREV (Raw Data)')

ax.set_zlim(0,0.5)




plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
## AVG LANDING RATE PLOT
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=4)

pcm = ax.scatter(df['RREV_trigger'],df['vz_d'],df['landing_rate'],c=df['vz_d'],cmap=cmap)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Vz_d')



ax.set_xlabel('RREV_trigger')
ax.set_ylabel('vz_d')
ax.set_zlabel('LR')


plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:

fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

pcm = ax.scatter(df['vx_d'],df['vz_d'],df['My_d'])



ax.set_xlabel('vx_d')
ax.set_ylabel('vz_d')
ax.set_zlabel('My_d [N*mm]')

ax.set_title('My_d vs IC (Raw Data)')

## I have more data for vz = (1.5,2.75) than vz = (3.00,4.0) so My_d differences are likely not a trend, just sparse data




plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:

## AVG LANDING RATE PLOT
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)
pcm = ax.scatter(df['vx_d'],df['vz_d'],-df['impact_eul'],c=df['landing_rate'],cmap=cmap)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Landing %')

ax.set_xlabel('vx_d')
ax.set_ylabel('vz_d')
ax.set_zlabel('Impact Angle')


ax.set_zlim(-70,180)
ax.set_zticks([0,60,120,180])

## Failed landings in corner
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
## AVG LANDING RATE PLOT
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=0,vmax=180)
ax.plot_trisurf(df_avg['vx_d'],df_avg['vz_d'],-df_avg['impact_eul','mean'],cmap=cmap)


ax.set_xlim(0,2.75)
ax.set_ylim(1.0,4.0)
ax.set_zlim(-70,180)
ax.set_zticks([0,60,120,180])


ax.set_xlabel('vx_d')
ax.set_ylabel('vz_d')
ax.set_zlabel('Impact Angle')

ax.set_title('Avg Impact Angle vs IC')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
## AVG LANDING RATE PLOT
fig = plt.figure()
ax = fig.add_subplot(111)


cmap = mpl.cm.Greys
norm = mpl.colors.Normalize(vmin=0,vmax=1)

ax.scatter(df['RREV_threshold'],df['My_d'],c=df['landing_rate'],cmap=cmap)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Landing %')

ax.grid()
ax.set_xlim(3,7)
ax.set_ylim(0,10)
ax.hlines(7.77,3,7)
ax.text(3.05,8.0,'Motors Maxed Out')

ax.set_xlabel('RREV_thr')
ax.set_ylabel('My_d [N*mm]')
ax.set_title('Policy: My_d vs RREV_thr')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
## AVG LANDING RATE PLOT
fig = plt.figure()
ax = fig.add_subplot(111)


cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=0,vmax=3)

ax.scatter(df['RREV_threshold'],df['My_d'],c=df['vz_d'],cmap=cmap)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Vz_d')

ax.grid()
ax.set_ylim(0,10)
ax.set_xlim(3,7)

ax.hlines(7.77,3,7)
ax.text(3.05,-8.0,'Motors Maxed Out')

ax.set_xlabel('RREV_thr')
ax.set_ylabel('My_d [N*mm]')
ax.set_title('Policy: My_d vs RREV_thr')
## As Vz increases RREV_trigger decreases to preserve time to contact. It also finds a host of My_d that work and they vary because of the robustness of the legs and the fact that landing success isn't deterministic.
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
## AVG LANDING RATE PLOT
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)

pcm = ax.scatter(df['RREV_threshold'],df['vx_d'],df['My_d'],c=df['landing_rate'],cmap=cmap)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Landing %')

ax.set_zlim(0,10)
ax.set_xlim(3,7)

ax.set_xlabel('RREV_trigger')
ax.set_ylabel('vx_d')
ax.set_zlabel('My_d')


plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
## AVG LANDING RATE PLOT
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)

pcm = ax.scatter(df['RREV_threshold'],df['landing_rate'],df['My_d'],c=df['landing_rate'],cmap=cmap)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Landing %')

ax.set_zlim(0,10)
ax.set_xlim(3,7)

ax.set_xlabel('RREV_trigger')
ax.set_ylabel('Landing_Rate')
ax.set_zlabel('My_d')


plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
## AVG LANDING RATE PLOT
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)

pcm = ax.scatter(df['RREV_trigger'],df['vz_d'],df['My_d'],c=df['landing_rate'],cmap=cmap)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Landing %')

ax.set_zlim(0,10)
ax.set_xlim(3,7)

ax.set_xlabel('RREV_trigger')
ax.set_ylabel('Vz_d')
ax.set_zlabel('My_d')


plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)

pcm = ax.scatter(df['vx_d'],df['My_d'],df['RREV_threshold'],c=df['landing_rate'],cmap=cmap)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Landing %')

# ax.set_xlim(3,7)
# ax.set_ylim(0,-10)
# ax.set_zlim(0,-10)

ax.set_xlabel('Vz_d')
ax.set_ylabel('OF_y_trigger')
ax.set_zlabel('RREV_threshold')


plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)

pcm = ax.scatter(df['G1'],df['OF_y'],df['My_d'],c=df['landing_rate'],cmap=cmap)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Landing Rate')

# ax.set_xlim(3,7)
# ax.set_ylim(0,-10)
ax.set_zlim(0,10)

ax.set_xlabel('G1')
ax.set_ylabel('OF_y')
ax.set_zlabel('My_d')


plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)

pcm = ax.scatter(df['RREV_threshold'],df['OF_y'],df['My_d'],c=df['landing_rate'],cmap=cmap)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Landing Rate')

ax.set_xlim(6,2)
ax.set_ylim(-10,0)
ax.set_zlim(0,10)

ax.set_xlabel('RREV')
ax.set_ylabel('OF_y')
ax.set_zlabel('My_d')


plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
df[df['RREV_threshold']<=3.5]

,vz_d,vx_d,trial_num,landing_rate,RREV_threshold,G1,RREV_sig,G1_sig,RREV_trigger,OF_y,My_d,impact_eul,impact_tdelta,alpha_mu,alpha_sigma,mu_ini,sigma_ini
244,2.50,1.50,3,0.500000,3.49,7.72,0.01,0.12,3.792000,-2.088000,7.633000,-103.596730,0.386000,[0. 0.],[0. 0.],[2.03 6.82],[1.5 1.5]
342,2.75,2.75,3,1.000000,3.43,6.90,0.00,0.01,3.731000,-3.456500,6.892500,-124.464691,0.402000,[0. 0.],[0. 0.],[2.02 6.84],[1.5 1.5]
377,3.00,1.25,2,0.857143,3.25,7.12,0.00,0.00,3.559000,-1.378500,7.118167,-70.107847,0.417333,[0. 0.],[0. 0.],[2. 4.96],[1.5 1.5]
379,3.00,1.25,4,0.666667,3.34,6.77,0.01,0.01,3.509500,-1.359500,6.777000,-106.404475,0.425000,[0. 0.],[0. 0.],[2.48 4.87],[1.5 1.5]
545,3.50,2.25,3,1.000000,3.19,6.31,0.00,0.06,3.383000,-2.046167,6.338333,-129.493273,0.451000,[0. 0.],[0. 0.],[1.36 5. ],[1.5 1.5]
551,3.50,2.50,4,1.000000,2.99,6.67,0.00,0.17,3.294667,-2.205000,6.703333,-112.332037,0.454000,[0. 0.],[0. 0.],[2.72 4.73],[1.5 1.5]
581,3.75,0.75,4,0.750000,3.48,2.42,0.00,0.01,3.680333,-0.701000,2.411333,-151.747757,0.352000,[0. 0.],[0. 0.],[2.9 2.18],[1.5 1.5]
598,3.75,1.50,4,0.400000,2.92,6.97,0.00,0.00,3.199500,-1.216500,6.967500,-120.943575,0.449000,[0. 0.],[0. 0.],[1.92 5.63],[1.5 1.5]
624,3.75,2.75,3,0.500000,3.12,4.41,0.00,0.11,3.430000,-2.401000,4.510000,-95.106556,0.502000,[0. 0.],[0. 0.],[2.19 1.79],[1.5 1.5]
667,4.00,1.75,3,1.000000,3.46,1.03,0.00,0.00,3.885000,-1.617000,1.024875,-137.536250,0.409000,[0. 0.],[0. 0.],[1.91 1.33],[1.5 1.5]


In [24]:
from sklearn import linear_model


df_reg = df.query(f"landing_rate>={0.8} & RREV_threshold>{3.5}")


X = df_reg[['RREV_threshold','OF_y']]
Y = df_reg['My_d']

reg = linear_model.LinearRegression(normalize=True)
reg.fit(X,Y)
intercept = reg.intercept_
coeffs = reg.coef_

print(intercept)
print(coeffs)



-12.186731606781834
[3.84957529 0.25684764]


In [25]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")


My_reg = intercept + coeffs[0]*df_reg['RREV_threshold'] + coeffs[1]*df_reg['OF_y']
ax.plot_trisurf(df_reg['RREV_threshold'],df_reg['OF_y'],My_reg,label='Linear_Regression',color='blue')


cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Landing Rate')

ax.scatter(df_reg['RREV_threshold'],df_reg['OF_y'],df_reg['My_d'],c=df_reg['landing_rate'],cmap=cmap,norm=norm)



ax.set_xlim(6,2)
ax.set_ylim(-10,0)
ax.set_zlim(0,10)


ax.set_xlabel('RREV')
ax.set_ylabel('OF_y')
ax.set_zlabel('My_d')


plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
def r_squared(y, y_hat):
    y_bar = y.mean()
    ss_tot = ((y-y_bar)**2).sum()
    ss_res = ((y-y_hat)**2).sum()
    return 1 - (ss_res/ss_tot)

R2 = r_squared(df_reg['My_d'],My_reg)
print(f"R^2: {R2:.3f}")

R^2: 0.463


In [27]:
import statsmodels.api as ssm #for detail description of linear coefficients, intercepts, deviations, and many more
X=ssm.add_constant(X)        #to add constant value in the model
model= ssm.OLS(Y,X).fit()         #fitting the model
predictions= model.summary()      #summary of the model
predictions


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   My_d   R-squared:                       0.463
Model:                            OLS   Adj. R-squared:                  0.456
Method:                 Least Squares   F-statistic:                     68.08
Date:                Fri, 05 Feb 2021   Prob (F-statistic):           4.75e-22
Time:                        11:33:41   Log-Likelihood:                -248.04
No. Observations:                 161   AIC:                             502.1
Df Residuals:                     158   BIC:                             511.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const            -12.1867      1.472     -8.277      0.000     -15.095      -9.279
RREV_threshold     3.8496      0.346     11.112      0.000       3.165       4.534
OF_y               0.2568      0.087      2.955      0.004       0.085       0.428
==============================================================================
Omnibus:                        0.196   Durbin-Watson:                   1.862
Prob(Omnibus):                  0.907   Jarque-Bera (JB):                0.331
Skew:                           0.065   Prob(JB):                        0.847
Kurtosis:                       2.819   Cond. No.                         102.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""